### Training with deeper CNN

In [33]:
import numpy as np
import torch as th
import torch.optim as optim

from torch import nn
from torch.utils.data import DataLoader
from cnn import CNN
from tqdm import tqdm
from imgdata import DefaultTrainSet, DefaultTestSet

In [34]:
%matplotlib inline

if th.cuda.is_available():
  # Make CuDNN Determinist
  th.backends.cudnn.deterministic = True

# Define default device, we should use the GPU (cuda) if available
device = th.device("cuda" if th.cuda.is_available() else "cpu")
device

device(type='cpu')

### Set constants

In [35]:
lr = 1e-3
epochs = 24
batch_size = 16
class_label = ['faces', 'dog', 'airplanes', 'keyboard', 'cars']

In [36]:
train_set = DefaultTrainSet()

train_dataloader = th.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)

### Define model

In [37]:
cnn = CNN()
fc_inputs = 64 * 30 * 30

In [38]:
cnn.fc = nn.Sequential(
    nn.Linear(fc_inputs, 64),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(64, 5)
)

In [39]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=lr)

### Train model

In [40]:
def train(_net, n_epochs, train_loader, _optimizer, _criterion):
    """
    Train a neural network and print statistics of the training

    :param _criterion:
    :param _optimizer:
    :param train_loader: loader for the training data
    :param _net: (PyTorch Neural Network)
    :param n_epochs: (int)  Number of iterations on the training set
    """
    print("---------- HYPERPARAMETERS ----------")
    print("n_epochs=", n_epochs)
    print("-" * 40)
    accuracies = []
    losses = []
    # Move model to gpu if possible
    _net = _net.to(device)
    for epoch_count, epoch in enumerate(range(n_epochs)):
        # Set the model to training mode
        _net.train()
        # Initialize the variables to compute the average loss
        total_loss = 0.00
        total_correct = 0
        pbar = tqdm(train_loader)
        # Iterate over the training set
        for i, data in enumerate(pbar):

            # Get the inputs
            inputs, labels = data['imNorm'], data['label']
            # Move them to the correct device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the parameter gradients
            _optimizer.zero_grad()

            # Forward pass
            outputs = _net(inputs)
             # Calculate Loss: softmax --> cross entropy loss
            loss = _criterion(outputs, labels)
            _, preds = th.max(outputs, 1)
            # Getting gradients w.r.t. parameters
            loss.backward()

            # Updating parameters
            _optimizer.step()

            # Update the running variables
            total_loss += loss.item()
            avg_loss = total_loss / (i + 1)
            total_correct += th.sum(preds == labels.data)
            pbar.set_description("Epoch: {}/{}".format(epoch_count + 1, n_epochs))
            pbar.set_postfix(avergae_loss=avg_loss, running_accuracy=th.sum(preds == labels).item() / float(labels.size(0)))
            # save the model
            model_name = f"Outputs/models/deeperCNN.pth"
            th.save(_net.state_dict(), model_name)

        # Calculate the average loss and accuracy of the epoch
        losses.append(total_loss/len(train_loader))
        accuracies.append((total_correct/train_loader.sampler.num_samples).item())
        # print losses and accuracies in the last epoch
        if epoch_count == n_epochs - 1:
            print("---------- FINAL RESULTS ----------")
            print("Losses fot every epoch:", losses)
            print("Average accuracy for each epoch: {}".format(accuracies))
            print("-" * 40)
    return losses, accuracies

In [41]:
losses, accuracies = train(cnn, epochs, train_dataloader, optimizer, loss_func)

---------- HYPERPARAMETERS ----------
n_epochs= 24
----------------------------------------


Epoch: 24/24: 100%|██████████| 22/22 [00:11<00:00,  1.85it/s, avergae_loss=0.277, running_accuracy=0.929]

---------- FINAL RESULTS ----------
Losses fot every epoch: [5.369963353330439, 0.9959360767494548, 0.7853192985057831, 0.6600350398908962, 0.6564003987745806, 0.4895555410872806, 0.4093782657926733, 0.43366342918439343, 0.39104930379173974, 0.3619141260331327, 0.32328908856619487, 0.29647820840843697, 0.33398534763943066, 0.3574068153446371, 0.41259434683756396, 0.3756222674115138, 0.3705765503374013, 0.2964827794242989, 0.26204031180929055, 0.24674516255882653, 0.21342520087703384, 0.33793801915916527, 0.26191372492096643, 0.2770949338816784]
Average accuracy for each epoch: [0.41428571939468384, 0.5914285778999329, 0.668571412563324, 0.7085714340209961, 0.7428571581840515, 0.7771428823471069, 0.8371428847312927, 0.8114285469055176, 0.8257142901420593, 0.8428571224212646, 0.8799999952316284, 0.8799999952316284, 0.8514285683631897, 0.8742856979370117, 0.8114285469055176, 0.831428587436676, 0.845714271068573, 0.8771428465843201, 0.8885714411735535, 0.9028571248054504, 0.920000016689300

### Evaluate model

In [42]:
# load test set and create dataloader
test_set = DefaultTestSet()
test_loader = DataLoader(test_set)
test_y = [i['label'] for i in test_set]

In [43]:
def predict(_model, _tests):
  predicts = []
  with th.no_grad():
    for i in tqdm(_tests):
        _model.eval()
        images = i['imNorm'].clone().to(device)
        outputs = _model(images)
        _, predicted = th.max(outputs.data, 1)
        predicts.append(predicted.item())
  return predicts

cnn.load_state_dict(th.load('Outputs/models/deeperCNN.pth'))
predict_y = predict(cnn, test_loader)

100%|██████████| 50/50 [00:01<00:00, 40.30it/s]


In [44]:
correct = []
incorrect = []
for i, j in enumerate(predict_y):
    if j == test_y[i]:
      correct.append(i)
    else:
      incorrect.append(i)

# generate confusion matrix
def confusion_matrix(y_true, y_pred):
  n_class = np.unique(y_true).shape[0]
  cm = np.zeros((n_class, n_class))
  for i in range(len(y_true)):
    cm[y_true[i], y_pred[i]] += 1
  return cm


cm = confusion_matrix(test_y, predict_y)
print(cm)

def print_accuracy(cm):
  for i in range(5):
    print(f'the accuracy by class {class_label[i]} : {cm[i, i] / np.sum(cm[i, :])}')
  # overall accuracy
  print(f'overall accuracy : {np.sum(cm[range(5), range(5)]) / np.sum(cm)}')

print_accuracy(cm)

[[ 3.  6.  0.  0.  1.]
 [ 0.  8.  2.  0.  0.]
 [ 0.  4.  6.  0.  0.]
 [ 0.  5.  0.  5.  0.]
 [ 0.  0.  0.  0. 10.]]
the accuracy by class faces : 0.3
the accuracy by class dog : 0.8
the accuracy by class airplanes : 0.6
the accuracy by class keyboard : 0.5
the accuracy by class cars : 1.0
overall accuracy : 0.64
